In [2]:
!pip install -q --upgrade pip
!pip install -q tifffile
!pip install -q librosa
!pip install -q tqdm
!pip install -q pysoundfile

# this is literally the hackiest piece of code I had ever written...
librosa_audio_path = '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/librosa/core/audio.py'
modified = open(librosa_audio_path).read().replace('import soundfile as sf', 'pass')
open(librosa_audio_path, 'w').write(modified);

In [3]:
import os
from glob import glob
import numpy as np
from tqdm import tqdm_notebook as tqdm
import librosa
import librosa.display
import wave
import scipy
import matplotlib.pyplot as plt
import imageio
import json
import pandas as pd

from fastai.vision import *

SPECTROGRAM_SCALE = 62.5 # how many frames / s (sr / hop_length)
SEGMENT_SIZE = 12 * SPECTROGRAM_SCALE

In [4]:
if not os.path.exists('data'):
    !mkdir data

if not os.path.exists('data/zealandia.tar.gz') and not os.path.exists('data/zealandia'):
    !curl https://datadryad.org/stash/downloads/file_stream/108711 -o data/zealandia.tar.gz

if not os.path.exists('data/zealandia'):
    !mkdir data/zealandia
    !tar -C data/zealandia -zxvf data/zealandia.tar.gz
    !rm data/zealandia.tar.gz

In [5]:
if not os.path.exists('spectrograms'):
    !mkdir spectrograms

for recording in tqdm(glob('data/zealandia/recordings1006/*.wav')):
    recording_name = recording.split('/')[-1]
    if os.path.exists(f'spectrograms/{recording_name}.tiff'):
        continue
    
    sr, y = scipy.io.wavfile.read(recording)
    y = y.astype(float)
    
    
    mel_spec = librosa.feature.melspectrogram(y, sr=sr, fmin=10, fmax=2000, power=1.0)
    mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)
    
    imageio.imwrite(f'spectrograms/{recording_name}.tiff', mel_spec_db)

In [60]:
labels = pd.DataFrame() if not os.path.exists('labels.csv') else pd.read_csv('labels.csv')

if not os.path.exists('segments'):
    !mkdir segments

for spectrogram in tqdm(glob('spectrograms/*.tiff')):
    recording_name = spectrogram.split('/')[-1].split('.')[0]
    spec = imageio.imread(spectrogram)
    
    n_segments = int(spec.shape[1] // SEGMENT_SIZE)
    if len(labels) and (labels.recording == recording_name).sum() >= n_segments:
            continue
        
    data_path = f'data/zealandia/manual1006/{recording_name}.wav.data'
    _, *data = json.load(open(data_path))
    
    is_kiwi = np.zeros(spec.shape[1], dtype=int)
    for label in data:
        t_start, t_end, _, _, species = label
        
        t_start_index = round(t_start * SPECTROGRAM_SCALE)
        t_end_index = round(t_end * SPECTROGRAM_SCALE)
        
        is_kiwi[t_start_index:t_end_index] = 1
    
    for i in range(n_segments):
        start_index = int(i * SEGMENT_SIZE)
        end_index = start_index + int(SEGMENT_SIZE)
        
        segment = spec[:, start_index:end_index]
        segment_image = 255 * ((segment + 80) / 80)
        
        segment_path = f'segments/{recording_name}_{i}.png'
        imageio.imwrite(segment_path, segment_image.astype(np.uint8))
        
        label = is_kiwi[start_index:end_index].mean()
        labels = labels.append({
            'recording': recording_name,
            'segment_index': i,
            'segment_path': segment_path,
            'is_kiwi': label >= 0.05
        }, ignore_index=True)

    labels.to_csv('labels.csv', index=False)

In [65]:
is_kiwi = ['yes' if label == 1 else 'no' for label in labels.is_kiwi.values]
data = ImageDataBunch.from_lists('', labels.segment_path.values, is_kiwi)

In [66]:
learn = cnn_learner(data, models.resnet50, metrics=accuracy)
learn.fit(4)

epoch,train_loss,valid_loss,accuracy,time
0,0.251894,0.243691,0.930117,10:24
1,0.227362,0.237163,0.932500,10:12


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 21, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 276, in rebuild_storage_fd
    fd = df.detach()
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_

KeyboardInterrupt: 